In [36]:
%reload_ext autoreload
%autoreload 2

In [67]:
import os
import pprint
import shutil

import pymongo
import pandas as pd
from IPython import display

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad


from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap

## Run clirad-lw

In [68]:
def get_params():
    '''
    Define the clirad-lw case(s) to be run by
    specifying one or more input parameter dictionary.
    '''
    
    param0 = {'commitnumber': 'a6676f4',
              'molecule': {'co2': 400e-6},
              'band': [4],
              'atmpro': 'mls',
              'tsfc': 294,}

    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]

    params = []
    for atmpro, tsfc in atmpros:
        param = param0.copy() 
        param['atmpro'] = atmpro
        param['tsfc'] = tsfc
        params.append(param)
        
    return params



def script():
    for param in get_params():
        try:
            fdir = pipeline.get_fortran_dir(param=param, setup=setup)
            shutil.rmtree(fdir)
        except FileNotFoundError:
            pass
        
        fproc = pipeline.run_fortran(param=param, setup=setup)

In [64]:
script()

{'atmpro': 'mls',
 'band': [4],
 'commitnumber': 'a6676f4',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumber': 'a6676f4',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': 'a6676f4',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}


## Save results in Mongodb

In [69]:
client = pymongo.MongoClient('localhost', 27017)
db = client.cliradnew
collection = db.lw

In [76]:
def make_doc(param=None):
    '''
    Make Mongodb document for a cliradlw run.
    
    Parameters
    ----------
    param: dict
        clirad-lw input parameters.
    doc: dict
        clirad-lw input (`param`) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_fortran_dir(param=param, 
                                           setup=setup)
    
    fnames = ['output_flux.dat', 'output_coolr.dat']
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname),
                  mode='r', encoding='utf-8') as f:
            s = f.read()
            
        s_outputs.append(s)
        
    doc = {fname.split('.')[0]: s
           for fname, s in zip(fnames, s_outputs)}
    doc['param'] = param
    return doc

In [72]:
params = get_params()

In [77]:
docs = (make_doc(param=param) for param in params)
#collection.insert_many(docs)

In [78]:
collection.count()

156